In [1]:
!pip install pytorch_pretrained_bert pytorch-nlp

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sys
import numpy as np
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

## Prepare the Data

In [4]:
train_data, test_data = imdb_dataset(train=True, test=True)
rn.shuffle(train_data)
rn.shuffle(test_data)
train_data = train_data[:1000]
test_data = test_data[:100]

aclImdb_v1.tar.gz: 84.1MB [00:10, 7.80MB/s]                            


In [42]:
len(train_data)

1000

In [43]:
train_data[0]

{'text': "This movie brings back many memories of the classic cinema of old, where actors didn't have to take their clothes off to make viewers watch their film.<br /><br />Firstly I think the main plus point of this movie is the amazing chemistry between Shahid and Amrita, it is definitely the making of the film.<br /><br />I have seen lots of comments regarding the film being sickly sweet and overly slushy. In response to this, I think to a certain degree this is a correct analysis, however considering this is a Barjatya film I think that compared to MPK, HAHK, HSSH and MPKDH, it has been toned down significantly. HSSH was almost unbearable to watch in some places.<br /><br />In this film however, when the sentimental moments come along, you find yourself smiling, wishing the budding couple all the best and hoping that nothing bad happens to them.<br /><br />Another major plus point is the performances of Shahid and Amrita. Both have acted very well, especially Shahid who looks great

In [5]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(1000, 1000, 100, 100)

In [11]:
type(train_texts), type(train_texts[0])

(tuple, str)

In [8]:
train_texts[0]

"This movie brings back many memories of the classic cinema of old, where actors didn't have to take their clothes off to make viewers watch their film.<br /><br />Firstly I think the main plus point of this movie is the amazing chemistry between Shahid and Amrita, it is definitely the making of the film.<br /><br />I have seen lots of comments regarding the film being sickly sweet and overly slushy. In response to this, I think to a certain degree this is a correct analysis, however considering this is a Barjatya film I think that compared to MPK, HAHK, HSSH and MPKDH, it has been toned down significantly. HSSH was almost unbearable to watch in some places.<br /><br />In this film however, when the sentimental moments come along, you find yourself smiling, wishing the budding couple all the best and hoping that nothing bad happens to them.<br /><br />Another major plus point is the performances of Shahid and Amrita. Both have acted very well, especially Shahid who looks great in the f

In [61]:
train_texts[1]

"A sober, reflexive piece, a little miniature which blossoms into a magnificent humane pictorial sequence which goes beyond a mere dramatization for the screen. This quiet little story will hold you enthralled - if you do not have too many problems with the various Spanish accents ranging from Mexican to Peruvian, and Marisa Paredes' more authentic Iberian Peninsular usage! Garcíadiego has accomplished a perfect adaptation from the novel: even the grand maestro García Márquez should be proud of her superb work. And hats off to Arturo Ripstein who has so ably concerted the whole effort into a gem, a ruby, and so refined, so elegant, so sensitive, so touchingly.....<br /><br />El Coronel - Fernando Luján - is waiting to get his pension, while he continues to live in his ramshackle timber dwelling deep in the Colombian jungle (however, filmed elsewhere, NOT in Colombia) with his fighting cock and his wife (in that order?). And that is all there is to it.<br /><br />But, oh, so much more..

In [10]:
len(train_texts)

1000

In [13]:
train_labels[0]

'pos'

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 1811388.87B/s]


In [15]:
tokenizer.tokenize('Hi my name is Dima')

['hi', 'my', 'name', 'is', 'dim', '##a']

In [16]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)                   
                   

(1000, 100)

In [84]:
type(train_tokens), type(train_tokens[0])

(list, list)

In [19]:
train_tokens[0]

['[CLS]',
 'this',
 'movie',
 'brings',
 'back',
 'many',
 'memories',
 'of',
 'the',
 'classic',
 'cinema',
 'of',
 'old',
 ',',
 'where',
 'actors',
 'didn',
 "'",
 't',
 'have',
 'to',
 'take',
 'their',
 'clothes',
 'off',
 'to',
 'make',
 'viewers',
 'watch',
 'their',
 'film',
 '.',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'firstly',
 'i',
 'think',
 'the',
 'main',
 'plus',
 'point',
 'of',
 'this',
 'movie',
 'is',
 'the',
 'amazing',
 'chemistry',
 'between',
 'shah',
 '##id',
 'and',
 'am',
 '##rita',
 ',',
 'it',
 'is',
 'definitely',
 'the',
 'making',
 'of',
 'the',
 'film',
 '.',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'i',
 'have',
 'seen',
 'lots',
 'of',
 'comments',
 'regarding',
 'the',
 'film',
 'being',
 'sick',
 '##ly',
 'sweet',
 'and',
 'overly',
 'sl',
 '##ush',
 '##y',
 '.',
 'in',
 'response',
 'to',
 'this',
 ',',
 'i',
 'think',
 'to',
 'a',
 'certain',
 'degree',
 'this',
 'is',
 'a',
 'correct',
 'analysis',
 ',',
 'however',
 'consid

In [20]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((1000, 512), (100, 512))

In [85]:
train_tokens_ids[0], train_tokens_ids[0].shape

(array([  101,  2023,  3185,  7545,  2067,  2116,  5758,  1997,  1996,
         4438,  5988,  1997,  2214,  1010,  2073,  5889,  2134,  1005,
         1056,  2031,  2000,  2202,  2037,  4253,  2125,  2000,  2191,
         7193,  3422,  2037,  2143,  1012,  1026,  7987,  1013,  1028,
         1026,  7987,  1013,  1028, 15847,  1045,  2228,  1996,  2364,
         4606,  2391,  1997,  2023,  3185,  2003,  1996,  6429,  6370,
         2090,  7890,  3593,  1998,  2572, 17728,  1010,  2009,  2003,
         5791,  1996,  2437,  1997,  1996,  2143,  1012,  1026,  7987,
         1013,  1028,  1026,  7987,  1013,  1028,  1045,  2031,  2464,
         7167,  1997,  7928,  4953,  1996,  2143,  2108,  5305,  2135,
         4086,  1998, 15241, 22889, 20668,  2100,  1012,  1999,  3433,
         2000,  2023,  1010,  1045,  2228,  2000,  1037,  3056,  3014,
         2023,  2003,  1037,  6149,  4106,  1010,  2174,  6195,  2023,
         2003,  1037,  3347,  3900, 21426,  2143,  1045,  2228,  2008,
      

In [22]:
train_y = np.array(train_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((1000,), (100,), 0.489, 0.5)

In [23]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [26]:
len(train_masks)

1000

In [28]:
train_masks[1]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

# Baseline

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [30]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

In [31]:
baseline_predicted = baseline_model.predict(test_texts)

In [32]:
print(classification_report(test_labels, baseline_predicted))

             precision    recall  f1-score   support

        neg       0.81      0.84      0.82        50
        pos       0.83      0.80      0.82        50

avg / total       0.82      0.82      0.82       100



# Bert Model

In [33]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
        

In [54]:
model = BertModel.from_pretrained('bert-base-uncased')

In [72]:
linear = nn.Linear(768,1)

In [56]:
x = next(iter(train_dataloader))

In [59]:
x

[tensor([[ 101, 2028, 1011,  ...,    0,    0,    0],
         [ 101, 2028, 1997,  ...,    0,    0,    0],
         [ 101, 2893, 2439,  ...,    0,    0,    0],
         [ 101, 2887, 3419,  ..., 2005, 1996,  102]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]]),
 tensor([[0.],
         [1.],
         [0.],
         [1.]])]

In [63]:
x[0].shape, x[1].shape, x[2].shape

(torch.Size([4, 512]), torch.Size([4, 512]), torch.Size([4, 1]))

In [87]:
x[0][0]

tensor([  101,  2028,  1011,  3602,  4038,  2008,  2763,  4520,  2715,  2154,
        10469,  2015,  1005,  4091,  2006,  3341,  1012,  2533,  3573,  7805,
        22396,  7867,  2003,  1999,  2293,  2007,  1996,  2801,  1997, 10834,
         1998,  3510,  1010, 22866,  2014,  8069,  2006,  2308,  1005,  1055,
         7298,  2127,  2016, 16794,  3565,  1011, 17395,  1999,  1996,  2433,
         1997, 21185,  5065, 20533,  3946,  1012,  1996,  2717,  1997,  1996,
         2143, 20228, 20620,  2013,  2028, 20228,  6977,  2000,  1996,  2279,
         2004,  1996, 21660,  7867,  7323,  2015,  2014, 12907,  1012,  1045,
         3984,  1996,  2773,  1000,  2566,  4801,  1000,  2074,  2055,  7680,
         1005,  1055,  2039,  7867,  1005,  1055,  3921,  2000,  1996,  2535,
         1012,  2016,  2515,  2031,  1037, 11951,  2868,  1010,  2021,  2044,
         2322,  2781,  1997, 24443, 21885,  2075,  2014, 21418, 14900,  1010,
         1045,  2211,  2000, 26641,  1012,  3946,  1005,  1055, 

In [66]:
y, pooled = model(x[0].long(), x[1].long(), output_all_encoded_layers=False)

In [68]:
y.shape

torch.Size([4, 512, 768])

In [69]:
pooled.shape

torch.Size([4, 768])

In [70]:
pooled[0].shape

torch.Size([768])

In [71]:
pooled[0]

tensor([ 6.2035e-01, -1.2935e-01, -6.3447e-01, -3.8974e-01,  4.2116e-01,
        -5.5024e-02, -9.3580e-01,  1.4767e-01, -5.9223e-01, -4.6779e-01,
         2.5108e-01,  3.9842e-01,  3.1596e-01,  5.0302e-01, -3.0484e-01,
         7.5943e-01,  8.6045e-01, -3.5990e-02,  6.9738e-03,  9.6940e-01,
         3.3081e-01,  9.9329e-01,  1.9115e-02,  3.4884e-01,  2.6564e-01,
         4.4706e-01,  2.5722e-02, -2.3618e-02, -1.0260e-01,  4.6844e-01,
         6.2713e-01, -3.7711e-02, -6.6075e-01, -2.5164e-01, -8.9383e-01,
        -2.2646e-01, -1.0696e-02,  4.0202e-01, -7.4280e-02,  3.4344e-01,
         2.4658e-01, -2.4652e-02,  8.9589e-01,  6.0270e-01, -3.4519e-02,
         1.0284e-01,  9.0841e-01,  1.4352e-01,  4.3273e-01,  7.6508e-01,
         4.6201e-01,  8.9849e-01, -1.2410e-01, -1.2699e-01, -5.5505e-02,
        -3.2303e-01, -2.6756e-01,  9.5763e-03, -4.8406e-02,  4.9331e-03,
        -3.1580e-01,  2.6484e-01, -6.0025e-01,  5.4551e-01,  1.9510e-01,
         7.8348e-01, -8.1185e-02,  4.8980e-02,  1.1

In [73]:
y[0].shape

torch.Size([512, 768])

In [74]:
z = linear(pooled)

In [75]:
z.shape

torch.Size([4, 1])

In [76]:
z[0]

tensor([-0.2232], grad_fn=<SelectBackward>)

In [79]:
sig = nn.Sigmoid()

In [82]:
prob = sig(z)

In [83]:
prob

tensor([[0.4444],
        [0.2773],
        [0.5422],
        [0.6100]], grad_fn=<SigmoidBackward>)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [35]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

ValueError: Expected a cuda device, but got: cpu

In [36]:
bert_clf = BertBinaryClassifier()
#bert_clf = bert_clf.cuda()


100%|██████████| 407873900/407873900 [00:20<00:00, 19801777.14B/s]


In [0]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.065088M'

In [37]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [38]:
y = bert_clf(x)
y.cpu().detach().numpy()

array([[0.431305 ],
       [0.4216192],
       [0.5204962]], dtype=float32)

In [0]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'6698.214912M'

In [40]:
y, x, pooled = None, None, None
#torch.cuda.empty_cache()
#str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

# Fine-tune BERT

In [41]:
BATCH_SIZE = 4
EPOCHS = 10

In [44]:
train_y.shape

(1000,)

In [45]:
train_y.reshape(-1,1).shape

(1000, 1)

In [46]:
torch.tensor(train_y.reshape(-1,1))

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True

In [47]:
torch.tensor(train_y.reshape(-1,1)).float()

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [48]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

#str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [49]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


In [50]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [51]:
param_optimizer

[]

In [52]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [0]:
 torch.cuda.empty_cache()

In [53]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        #print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
        

Epoch:  1
23/250.0 loss: 0.6890591283639272 


KeyboardInterrupt: 

In [0]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
    

In [33]:
np.mean(bert_predicted)

0.56

In [34]:
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.86      0.76      0.81        50
        True       0.79      0.88      0.83        50

    accuracy                           0.82       100
   macro avg       0.82      0.82      0.82       100
weighted avg       0.82      0.82      0.82       100

